## TensorFlow变量管理

选择环境：Anaconda Python 3.5.2  
安装Tensorflow：Python 3.5环境下运行pip install --upgrade --ignore-installed tensorflow  
参考书籍：《TensorFlow实战Google深度学习框架（第2版）》  
ipynb格式：点击阅读原文github

### 5.3 变量管理

TensorFlow提供了通过变量名称来创建或者获取一个变量的机制，主要通过tf.get_variable和tf.variable_scope实现。

In [ ]:
# 当tf.get_variable用于创建变量时，和tf.Variable功能基本等价
v = tf.get_variable("v", shape=[1],
                   initializer=tf.constant_initializer(1.0))
v = tf.Variable(tf.constant(1.0, shape=[1]), name="v")

In [1]:
import tensorflow as tf

# var1是一个数值
var1 = tf.get_variable("var1", shape=None, initializer=tf.constant(1.0))
# var2是一个一维数组
var2 = tf.Variable(tf.constant(1.0, shape=[1], dtype=tf.float32), name="var2")

# 即使设置了相同的name，实际上也是不同的变量
var3 = tf.Variable(tf.constant(1.0, shape=[1], dtype=tf.float32), name="var")
var4 = tf.Variable(1.0, name="var")
print("var1.name:", var1.name)
print("var2.name:", var2.name)
print("var3.name:", var3.name)
print("var4.name:", var4.name) # 自动变成var_1

# TF提供的变量初始化函数
var5 = tf.Variable(tf.random_normal(shape=[2, 3], mean=0.5, stddev=1.0, seed=1.0), name='var5')
var6 = tf.Variable(tf.ones(shape=[2, 2]), name='var6')
var7 = tf.Variable(tf.zeros(shape=[2, 1]), name='var7')

with tf.Session() as sess:
    #变量赋值函数，会自动创建一个operation，必须执行operation，否则不会进行赋值
    # var1是一个数值
    assign_op1 = tf.assign(var1, 2.5)
    # var2是一个一维数组
    assign_op2 = var2.assign([3.5])

    sess.run(tf.global_variables_initializer())
    print("before update var1:", var1.eval())
    sess.run(assign_op1)
    print("after update var1:", var1.eval())
    print("before update var2:", var2.eval())
    sess.run(assign_op2)
    print("after update var2:", var2.eval())
    print("var5:", var5.eval())
    print("var6:", var6.eval())
    print("var7:", var7.eval())

var1.name: var1:0
var2.name: var2:0
var3.name: var:0
var4.name: var_1:0
before update var1: 1.0
after update var1: 2.5
before update var2: [1.]
after update var2: [3.5]
var5: [[-0.31131822  1.9845988   0.5653294 ]
 [-1.9427042   0.5992484   1.0912243 ]]
var6: [[1. 1.]
 [1. 1.]]
var7: [[0.]
 [0.]]


对于tf.get_variable函数，变量名称是一个必填的参数，如果创建了同名的参数就会报错。

tf.variable_scope函数生成上下文管理器，控制tf.get_variable函数获取已经创建过的变量。

1. 上下文管理器foo中创建变量

In [2]:
# 在名字为foo的命名空间内创建名字为v的变量
with tf.variable_scope("foo"):
    v = tf.get_variable(
        "v", shape=[1], initializer=tf.constant_initializer(1.0))

# 因为在命名空间foo中已经存在名字为v的变量，所以以下代码将会报错
#with tf.variable_scope("foo"):
   # v = tf.get_variable("v", [1])

# 生成上下文管理器时，将参数reuse设置为True。这样tf.get_variable将直接获取已经声明的变量
with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", shape=[1])
print(v == v1) # 若输出为True，则代表v,v1是相同变量
print("v.name", v.name)
print("v1.name", v1.name)

# 如果命名空间中没有创建变量，reuse设置为True会报错，设置为False将创建新的变量，同名将报错
with tf.variable_scope("bar", reuse=False):
    v = tf.get_variable("v2", shape=[1])
print("v.name", v.name)

True
v.name foo/v:0
v1.name foo/v:0
v.name bar/v2:0


2. 嵌套上下文管理器中reuse参数的使用

In [3]:
with tf.variable_scope("root"):
    print(tf.get_variable_scope().reuse) # 获取reuse参数的取值
    
    with tf.variable_scope("foo", reuse=True):
        print(tf.get_variable_scope().reuse)
        
        with tf.variable_scope("bar"): # 不指定reuse
            print(tf.get_variable_scope().reuse) # 此时reuse取值和外面一层保持一致
            
    print(tf.get_variable_scope().reuse) # 退出上下文之后reuse又回到了False

False
True
True
False


3. 通过tf.variable_scope来管理变量的名称

In [31]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import tensorflow as tf
v1 = tf.get_variable("v1", [1])
print("v1.name:", v1.name)

with tf.variable_scope("foo"):
    v2 = tf.get_variable("v", [1])
    print("v2.name:", v2.name)

# 变量范围嵌套
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v3 = tf.get_variable("v", [1])
        print("v3.name:", v3.name)
    
    v4 = tf.get_variable("v_1", [1])
    print("v4.name:", v4.name)

v1.name: v1:0
v2.name: foo/v:0
v3.name: foo/bar/v:0
v4.name: foo/v_1:0


4. 通过变量名称获取变量

In [6]:
with tf.variable_scope("",reuse=True): # 创建名称为空的命名空间
    v5 = tf.get_variable("foo/bar/v", [1])
    print("v5 == v3:", v5 == v3)
    v6 = tf.get_variable("foo/v_1", [1])     
    print("v6 == v4:", v6 == v4)

v5 == v3: True
v6 == v4: True


通过tf.variable_scope和tf.get_variable函数，以下代码对MNIST数字识别模型中定义的计算前向传播结果的函数做了一些改进：

In [ ]:
def inference(input_tensor, reuse = False):
    # 定义第一层神经网络的变量和前向传播结果
    with tf.variable_scope('layer1', reuse = reuse):
        # 根据传进来的`reuse`来判断是创建新的变量还是使用已经创建好了的。
        # 在第一次构造网络时需要创建新的变量，
        # 以后每次调用这个函数都直接使用 `reuse = True`就不需要每次将变量传进来了
        weights = tf.get_variable('weights', [INPUT_NODE, LAYER1_NODE], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [LAYER1_NODE], 
                                 initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
    
    # 类似地定义第二层神经网络的变量和前向传播结果
    with tf.variable_scope('layer2'):
        weights = tf.get_variable("weights", [LAYER1_NODE, OUTPUT_NODE], 
                                  initializer=tf.truncated_normal_initializer(stddev=0.1))
        biases = tf.get_variable("biases", [OUTPUT_NODE], 
                                 initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
    # 返回最后的前向传播结果
    return layer2

x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
y = inference(x)

# 在程序中需要使用训练好的神经网络进行推导时，直接调用inference(new_x,True)
new_x = ...
new_y = inference(new_x,True)

使用这种方式就不再需要将所有变量都作为参数传递到不同的函数中了，大大提高程序的可读性。